# Repository Footprint Analysis

Computes repository foot print for each reactor, including storage time and number of fuel elements per package. This is based on the NWPY package by Atz (2019)

**Note:** This script runs on a different version of Python from the rest of R2R4SNF as the nwpy package was designed in 2019. Please downgrade to Python version <=2.7 to run this script.

## Import and Setup


In [1]:
# %matplotlib inline
import nwpy
import pandas as pd
from nwpy import fuelcycle
from nwpy.fuelcycle import stage
from nwpy.plot import TimePlot as tp
from nwpy.repository_area import repository
from nwpy.repository_area import waste
from nwpy import stream
import math
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
def setup(reactor, workspace_path):
    '''Read in reactor, canister, and time-independent parameters from OpenMC simulation.'''
    ## ---- Parameters:
    # reactor (str) - reactor name
    # workspace_path (str) - path to the R2R4SNF workspace
    ## ---- Returns: 
    # parameters (dict) - a dictionary containing three nested dicts for reactor, canister, and time-independent parameters
    
    # Reading in reactor parameters as dict.
    reactors = pd.read_csv(workspace_path + 'inputs/parameters/reactor_parameters.csv', index_col = 0).T.to_dict()

    # Reading in canister parameters as dict. Group by fuel type, convert the inner dataframe to 
    # a dictionary, and nest into a higher-level dict for fuel type.
    canisters = pd.read_csv(workspace_path + 'inputs/parameters/canister_parameters.csv', index_col=[0,1])
    canister_parameters_all = canisters.groupby(level=0).apply(lambda df: df.xs(df.name).T.to_dict()).to_dict()
    
    # Reading in time-independent parameters as dict.
    time_ind_parameters = pd.read_csv((workspace_path + 'results/' + reactor + '/reactor_simulation/summary/out_' + reactor +
                                       '_time_ind.csv'), index_col = 0, header=None, squeeze=True).to_dict()
    
    # Build parameters dict
    parameters = {}
    parameters['reactor'] = reactors[reactor]
    parameters['canister'] = canister_parameters_all[parameters['reactor']['fuel_type']]
    parameters['time_ind'] = time_ind_parameters
    
    return(parameters)

## Build Functions

In [3]:
def build_nwpy_wasteform(reactor, parameters, n_wf, workspace_path):
    """Creates the input data used to build a Waste form in nwpy."""
    
    ## ---- Parameters: 
    # reactor (str) - reactor name
    # parameters (dict) - input parameters for each reactor and canister type
    # n_wf (int) - the number of fuel units per canister according to the technology of reactor
    # workspace_path (str) - path to the R2R4SNF workspace"""    
    
    # Create header data for nwpy waste form
    header = {'name': 'uox', 
              'id':'snf', 
              'pkg_diameter': parameters['canister'][n_wf]['Diameter'],
              'pkg_length': parameters['canister'][n_wf]['Length'],
              'n_wf':n_wf}
    header_df = pd.DataFrame(sorted(header.items()), columns = ['Years', 'Waste'])

    # path to the cooling_reactor dataset, measuring reactor simulation from the step power runs out
    result    = pd.read_csv(workspace_path + "results/" + reactor + "/reactor_simulation/summary/cooling_" + 
                            reactor + ".csv", sep=',') 

    # to exclude first row, result.loc[1:, ['Days', 'Decay heat (W/g)']]
    decay_heat = pd.DataFrame()
    decay_heat['Years'] = (result['Days'] - result['Days'].iloc[0]) / 365
    decay_heat['Waste'] = result['Decay heat (W/g)'] * float(parameters['reactor']['hm_per_asm']) * n_wf
    
    
    # combine the two datasets and write to csv
    path = (workspace_path + "results/" + reactor + "/repository_simulation/nwpy_wasteforms/" + 
            reactor + "_waste_N" + str(n_wf) + '.csv')
    nwpy_wasteform = pd.concat([header_df, decay_heat]).set_index('Years')
    nwpy_wasteform.to_csv(path, index=True, header=True)

In [4]:
def repository_instance(rock_type, reactor, storagetime, n_wf, workspace_path):
    """
    Create a repository instance in the 'rock_type' evironment, with a specific storagetime before disposal, containing the nuclear waste from the 'reactor' reactor.
    The repository contains identical packages with 'nf_per_can' waste units per canister and identical surface storage storage before disposal.
    """
    ## ---- Parameters:
    # rock_type - repository rock type
    # reactor (str) - reactor name
    # storagetime (float) - storage time in years
    # n_wf (int) - the number of fuel unit per canister according to the technology of reactor
    # workspace_path (str) - a path to the directory in which openmc files are stored       
    ## ---- Returns: 
    # rep (Object) - repository instance in nwpy
    
    
    path = (workspace_path + 'results/' + reactor + '/repository_simulation/nwpy_wasteforms/' +
            reactor + '_waste_N' + str(n_wf) + '.csv')
    wf   = waste.Waste(path)
    
    rep = repository.Repository(rock_type, st=storagetime)
    rep.load_waste(wf)
    
    return rep

In [5]:
def calculate_app(rep, st_time, n_wf):
    """
    Returns a list of area per package for a specific reactor and canister design according to the surface storage time.
    A list of surface storage time is given in input
    """
    ## ---- Parameters:
    # rep (Object) - repository instance in nwpy
    # st_time (list) - list of storage times in years
    # n_wf (int) - the number of fuel units per canister according to the technology of reactor       
    ## ---- Returns:
    # apps (list) - list of area per package numbers for different storage times
    
    apps = [] # list containing the area per package
    for storagetime in st_time:
        rep.update_st(storagetime) # Update the surface storage time
        app = rep.calculate_app(log=False)
        apps.append(app)
    return apps   

In [6]:
def calculate_N_pkg(reactor, parameters, n_wf): 
    """Calculate the number of package produced per 100 GWe.year"""
    ## ---- Parameters:
    # reactor (str) - reactor name
    # parameters (dict) - input parameters for each reactor and canister type
    # n_wf (int) - the number of fuel unit per canister according to the technology of reactor"""        
    ## ---- Returns:
    # N_pkg - the number of canisters in the repository   
    
    hm_mass  = float(parameters['reactor']['hm_per_asm']) * n_wf
    SNF_mass = parameters['time_ind']['SNF mass (t/GWe.y)']
    
    elec_per_pkg = hm_mass * (10**-6) / SNF_mass # electricity produced per waste package 
    N_pkg        = 1 / elec_per_pkg
    
    return N_pkg

In [7]:
def build_repository_metrics(rock_type, reactor, parameters, st_time, n_wf, workspace_path):
    """
    Generate a dataframe for each number of package per canister according to the reactor technology.
    """
    ## ---- Parameters:
    # rock_type (str) - repository rock type
    # reactor (str) - reactor name
    # parameters (dict) - input parameters for each reactor and canister type
    # st_time (float) - storage time in years
    # n_wf (int) - the number of fuel units per canister according to the technology of reactor       
    # primary_path (str) - a path to the R2R4SNF directory

    ## ---- Returns:
    # rep_footprint (df) - dataframe containing the repository footprint over time for reactor with a given number of fuel units
    
    # Create the Waste input file necessary for app calculation
    build_nwpy_wasteform(reactor, parameters, n_wf, workspace_path) 

    # Create the Repository instance, reading in the Waste input file
    rep = repository_instance(rock_type, reactor, st_time, n_wf, workspace_path) 

    # Calculate number of packages and area per package using the Repository instance
    N_pkg = calculate_N_pkg(reactor, parameters, n_wf)
    apps = calculate_app(rep, st_time, n_wf)

    # Build repository footprint dataframe
    rep_footprint = pd.DataFrame(index = st_time)
    rep_footprint['#packages per GWe.y'] = N_pkg
    rep_footprint['Area per package (m2)'] = apps
    rep_footprint['Total area (m2)'] = [N_pkg * app for app in apps]
    rep_footprint['Required st (y)'] = rep.calculate_required_st()

    return rep_footprint

## Output functions

In [8]:
def run_simulation(rock_type, reactor, st_time, workspace_path, output = False):
    ''' Load parameters, build repository footprint metrics, and write to csv.'''
    
    ## ---- Parameters:
    # rock_type (str) - repository rock type
    # reactor (str) - reactor name
    # st_time (float) - storage time in years
    # workspace_path (str) - a path to the R2R4SNF directory
    # output (bool) - a flag for whether the function should print output
    
    if output: print(reactor + " " + rock_type + " repository simulation ----------------")
    
    # Get reactor, canister, and time-independent parameters from OpenMC simulation
    parameters = setup(reactor, workspace_path)
    
    # Get the list of fuel unit setups for the reactor's canister type
    n_wfs = sorted(parameters['canister'].keys())
    
    # Calculate repository footprint for each fuel unit arrangement
    for n_wf in n_wfs:
        rep_footprint = build_repository_metrics(rock_type, reactor, parameters, st_time, n_wf, workspace_path)
        rep_footprint.to_csv(workspace_path + 'results/' + reactor + '/repository_simulation/footprints/' + 
                             reactor + '_' + rock_type + '_footprint_N' + str(n_wf) + '.csv')
        
        if output: print(str(n_wf) + " fuel unit footprint simulated")
            
    if output: print("\n")

## Post-processing functions

After all nwpy Waste forms have been built, we reassemble them to find minimum repository metrics for each reactor across all storage times.

In [9]:
def read_files(reactor, workspace_path):
    '''Read and combine all nwpy outputs for minimization comparisons'''
    
    ## ---- Parameters:
    # reactor (str) - reactor name
    # workspace_path (str) - path to the R2R4SNF workspace
    ## ---- Returns:
    # results (df) - a dataframe containing all nwpy footprint simulations and decay heats for the given reactor type
    
    # extracting the number of fuel canister arrangements using nwpy footprint simulation filenames
    footprint_path = workspace_path + 'Results/' + reactor + '/repository_simulation/footprints/'
    filenames = [f for f in os.listdir(footprint_path) if '.csv' in f and 'combined_footprints' not in f]

    flist = []
    for filename in filenames:
        footprint = pd.read_csv(footprint_path + filename, index_col = False).rename(columns = {'Unnamed: 0': 'Years'})
        n_wf = int(re.search(r'\d+', filename).group()) # extract n_wf number from filename using regex
        footprint['n_wf'] = n_wf

        decay_heat = pd.read_csv(workspace_path + 'Results/' + reactor+ '/repository_simulation/nwpy_wasteforms/' + \
                             reactor + '_waste_N' + str(n_wf) + '.csv', skiprows = range(1, 6), index_col = False)

        footprint['Decay heat (W/pkg)'] = [decay_heat.loc[decay_heat['Years'] > st, 'Waste'].iloc[0] for st in footprint['Years']]
        footprint['Reactor'] = reactor
        footprint.columns = ['Years', '#packages per GWe.y', 'Area per package (m2)', 'Total area (m2)', 'Required st (y)',
                          'n_wf', 'Decay heat (W/Pkg)', 'Reactor']
        
        flist.append(footprint)
    
    results = pd.concat(flist, axis=0, ignore_index=True)
    results.to_csv(footprint_path + reactor + '_combined_footprints.csv')
    
    return results

In [10]:
def get_min_areas(reactors, workspace_path, st):
    """
    Returns a dataframe with the minimum area for each reactor type given a surface storage time
    """

    ## ---- Parameters:
    # reactor (str) - reactor name
    # workspace_path (str) - path to the R2R4SNF workspace
    ## ---- Returns:
    # df_min (df) - dataframe containing the minimum repository size for each reactor at time st
    
    flist = []
    for reactor in reactors:
        results = read_files(reactor, workspace_path)

        filter_df = results[(results['Years'] == st) & (results['Total area (m2)'] > 0)]

        if (len(filter_df) != 0):
            min_values = filter_df.iloc[[filter_df['Total area (m2)'].argmin()]]
            flist.append(min_values)

    df_min = pd.concat(flist, axis=0, ignore_index=True)
    
    return df_min

# Example

In [11]:
# Set up workspace
workspace_path = '/Users/krishnasunder/Desktop/workspace/R2R4SNF/'
reactors = ['Ref_PWR', 'SPWR', 'HTGR', 'HPR', 'HTGR_FCM', 'SFR']
sts = [3.0, 5.0, 10.0, 20.0, 50.0, 100.0]
rock_type = 'clay'
reactor = 'SFR'

In [12]:
# Run repository simulation to create nwpy Waste forms for each reactor
for reactor in reactors:
    run_simulation(rock_type, reactor, sts, workspace_path, True)

Ref_PWR clay repository simulation ----------------
No solution; single package violates constraint
No solution; single package violates constraint
[10. 10.]
iter 1;   DS = 10.000  WPS = 10.000  A = 6984.620   WP Peak Temp = 88.764
[10. 10.]
iter 2;   DS = 10.000  WPS = 10.000  A = 6984.620   WP Peak Temp = 88.764
[11. 10.]
iter 3;   DS = 11.000  WPS = 10.000  A = 7664.620   WP Peak Temp = 87.306
[10. 11.]
iter 4;   DS = 10.000  WPS = 11.000  A = 7641.996   WP Peak Temp = 87.306
[9.29289322 9.29289322]
iter 5;   DS = 9.293  WPS = 9.293  A = 6070.952   WP Peak Temp = 91.655
[8.59827758 8.57351216]
iter 6;   DS = 8.598  WPS = 8.574  A = 5221.680   WP Peak Temp = 95.336
[8.05252818 7.73556371]
iter 7;   DS = 8.053  WPS = 7.736  A = 4453.988   WP Peak Temp = 99.814
[7.66142107 8.04706849]
iter 8;   DS = 7.661  WPS = 8.047  A = 4402.869   WP Peak Temp = 100.107
[8.38103237 7.35269138]
iter 9;   DS = 8.381  WPS = 7.353  A = 4417.743   WP Peak Temp = 100.097
[7.87537068 7.87537068]
iter 10;  

iter 4;   DS = 12.793  WPS = 13.793  A = 12103.203  WP Peak Temp = 74.865
[12.08624401 12.08624401]
iter 5;   DS = 12.086  WPS = 12.086  A = 10094.960  WP Peak Temp = 77.687
[11.39030699 11.36814119]
iter 6;   DS = 11.390  WPS = 11.368  A = 8990.486   WP Peak Temp = 79.820
[10.80656002 10.55620555]
iter 7;   DS = 10.807  WPS = 10.556  A = 7964.103   WP Peak Temp = 82.361
[10.90719373  9.56128201]
iter 8;   DS = 10.907  WPS = 9.561  A = 7321.048   WP Peak Temp = 84.317
[10.18550159  8.86906783]
iter 9;   DS = 10.186  WPS = 8.869  A = 6385.027   WP Peak Temp = 87.757
[9.52880455 8.11491334]
iter 10;  DS = 9.529  WPS = 8.115  A = 5510.511   WP Peak Temp = 91.872
[9.0295419  7.24846265]
iter 11;  DS = 9.030  WPS = 7.248  A = 4712.635   WP Peak Temp = 96.718
[8.15700561 6.75991334]
iter 12;  DS = 8.157  WPS = 6.760  A = 4007.825   WP Peak Temp = 102.360
[8.96571494 6.17170485]
iter 13;  DS = 8.966  WPS = 6.172  A = 4040.381   WP Peak Temp = 101.865
[9.46529786 6.15128669]
iter 14;  DS = 9.4

iter 11;  DS = 9.045  WPS = 7.266  A = 4731.101   WP Peak Temp = 96.018
[8.17338096 6.77629541]
iter 12;  DS = 8.173  WPS = 6.776  A = 4024.472   WP Peak Temp = 101.400
[8.8697701  6.05863103]
iter 13;  DS = 8.870  WPS = 6.059  A = 3932.122   WP Peak Temp = 101.997
[9.07348164 6.34061346]
iter 14;  DS = 9.073  WPS = 6.341  A = 4188.833   WP Peak Temp = 99.825
[9.79663443 5.64992537]
iter 15;  DS = 9.797  WPS = 5.650  A = 4063.896   WP Peak Temp = 100.483
[10.24665169  5.43201606]
iter 16;  DS = 10.247  WPS = 5.432  A = 4097.218   WP Peak Temp = 99.985
[10.64633406  5.13159302]
iter 17;  DS = 10.646  WPS = 5.132  A = 4041.251   WP Peak Temp = 100.183
[10.88887295  5.        ]
iter 18;  DS = 10.889  WPS = 5.000  A = 4036.184   WP Peak Temp = 100.079
[11.02841185  5.20743407]
iter 19;  DS = 11.028  WPS = 5.207  A = 4237.114   WP Peak Temp = 98.532
[10.99258998  4.93023055]
iter 20;  DS = 10.993  WPS = 4.930  A = 4022.997   WP Peak Temp = 100.118
[10.90631531  5.02592926]
iter 21;  DS = 10

/Users/krishnasunder/miniconda3/envs/nwpy-env/lib/python2.7/site-packages/nwpy/repository_area/heat.py:389: RuntimeWarning: Integral not converged after 250 subdivisions; +abserr = 0.000149086282047
  'abserr = '+str(abserr),RuntimeWarning)


21 fuel unit footprint simulated


SPWR clay repository simulation ----------------
[10. 10.]
iter 1;   DS = 10.000  WPS = 10.000  A = 6984.620   WP Peak Temp = 48.554
[10. 10.]
iter 2;   DS = 10.000  WPS = 10.000  A = 6984.620   WP Peak Temp = 48.554
[11. 10.]
iter 3;   DS = 11.000  WPS = 10.000  A = 7664.620   WP Peak Temp = 48.538
[10. 11.]
iter 4;   DS = 10.000  WPS = 11.000  A = 7641.996   WP Peak Temp = 48.538
[9.29289322 9.29289322]
iter 5;   DS = 9.293  WPS = 9.293  A = 6070.952   WP Peak Temp = 48.594
[8.59827758 8.57351216]
iter 6;   DS = 8.598  WPS = 8.574  A = 5221.680   WP Peak Temp = 48.688
[8.05252818 7.73556371]
iter 7;   DS = 8.053  WPS = 7.736  A = 4453.988   WP Peak Temp = 48.910
[7.20492812 7.20492812]
iter 8;   DS = 7.205  WPS = 7.205  A = 3746.553   WP Peak Temp = 49.606
[6.63105071 6.3859869 ]
iter 9;   DS = 6.631  WPS = 6.386  A = 3097.196   WP Peak Temp = 51.874
[5.81210949 5.81210949]
iter 10;  DS = 5.812  WPS = 5.812  A = 2506.295   WP Peak Temp = 54.538
[5.2

...

iter 3;   DS = 14.776  WPS = 4.888  A = 5323.751   WP Peak Temp = 79.166
[13.77648996  5.8875    ]
iter 4;   DS = 13.776  WPS = 5.888  A = 5876.081   WP Peak Temp = 77.235
[12.77648996  4.8875    ]
iter 5;   DS = 12.776  WPS = 4.888  A = 4619.951   WP Peak Temp = 82.907
[11.77648996  4.8875    ]
iter 6;   DS = 11.776  WPS = 4.888  A = 4268.051   WP Peak Temp = 85.240
[10.77648996  4.8875    ]
iter 7;   DS = 10.776  WPS = 4.888  A = 3916.151   WP Peak Temp = 87.970
[9.77648996 4.8875    ]
iter 8;   DS = 9.776  WPS = 4.888  A = 3564.251   WP Peak Temp = 91.185
[8.77648996 4.8875    ]
iter 9;   DS = 8.776  WPS = 4.888  A = 3212.351   WP Peak Temp = 95.000
[7.77648996 4.8875    ]
iter 10;  DS = 7.776  WPS = 4.888  A = 2860.451   WP Peak Temp = 99.569
[7.77648996 4.3875    ]
iter 11;  DS = 7.776  WPS = 4.388  A = 2600.336   WP Peak Temp = 102.999
[7.01138552 5.53140621]
iter 12;  DS = 7.011  WPS = 5.531  A = 2894.660   WP Peak Temp = 99.518
[7.69600168 4.79186195]
iter 13;  DS = 7.696  WPS 

iter 5;   DS = 14.845  WPS = 14.845  A = 15065.686  WP Peak Temp = 81.396
[14.14928907 14.12763789]
iter 6;   DS = 14.149  WPS = 14.128  A = 13709.282  WP Peak Temp = 83.364
[13.54350769 13.33200675]
iter 7;   DS = 13.544  WPS = 13.332  A = 12428.647  WP Peak Temp = 85.567
[13.37805069 12.34578974]
iter 8;   DS = 13.378  WPS = 12.346  A = 11412.338  WP Peak Temp = 87.617
[12.63054399 11.68153547]
iter 9;   DS = 12.631  WPS = 11.682  A = 10239.183  WP Peak Temp = 90.409
[11.9820013  10.92035718]
iter 10;  DS = 11.982  WPS = 10.920  A = 9126.523   WP Peak Temp = 93.606
[11.59457281  9.99845743]
iter 11;  DS = 11.595  WPS = 9.998  A = 8133.814   WP Peak Temp = 97.046
[10.80894733  9.37975507]
iter 12;  DS = 10.809  WPS = 9.380  A = 7157.147   WP Peak Temp = 101.208
[11.56743866  8.72807194]
iter 13;  DS = 11.567  WPS = 8.728  A = 7143.641   WP Peak Temp = 101.137
[12.46605962  8.28934608]
iter 14;  DS = 12.466  WPS = 8.289  A = 7319.919   WP Peak Temp = 100.110
[13.24509729  7.66236897]
i

iter 4;   DS = 10.000  WPS = 11.000  A = 7645.716   WP Peak Temp = 46.237
[9.29289322 9.29289322]
iter 5;   DS = 9.293  WPS = 9.293  A = 6074.126   WP Peak Temp = 47.856
[8.59845254 8.57334326]
iter 6;   DS = 8.598  WPS = 8.573  A = 5224.631   WP Peak Temp = 49.105
[8.05299436 7.73520522]
iter 7;   DS = 8.053  WPS = 7.735  A = 4456.722   WP Peak Temp = 50.592
[7.20507692 7.20507692]
iter 8;   DS = 7.205  WPS = 7.205  A = 3749.205   WP Peak Temp = 52.398
[6.63106474 6.38623016]
iter 9;   DS = 6.631  WPS = 6.386  A = 3099.554   WP Peak Temp = 54.668
[5.81221798 5.81221798]
iter 10;  DS = 5.812  WPS = 5.812  A = 2508.442   WP Peak Temp = 57.525
[5.22886392 5.        ]
iter 11;  DS = 5.229  WPS = 5.000  A = 1981.931   WP Peak Temp = 61.307
[4.87734126 5.35557252]
iter 12;  DS = 4.877  WPS = 5.356  A = 1972.667   WP Peak Temp = 61.309
[5. 5.]
iter 13;  DS = 5.000  WPS = 5.000  A = 1899.540   WP Peak Temp = 62.031
[5.     4.9375]
iter 14;  DS = 5.000  WPS = 4.938  A = 1878.434   WP Peak Temp

iter 10;  DS = 6.873  WPS = 6.873  A = 3438.154   WP Peak Temp = 100.043
[6.87864278 6.87864278]
iter 1;   DS = 6.879  WPS = 6.879  A = 3443.340   WP Peak Temp = 95.984
[6.87864278 6.87864278]
iter 2;   DS = 6.879  WPS = 6.879  A = 3443.340   WP Peak Temp = 95.984
[7.87864278 6.87864278]
iter 3;   DS = 7.879  WPS = 6.879  A = 3923.573   WP Peak Temp = 92.006
[6.87864278 7.87864278]
iter 4;   DS = 6.879  WPS = 7.879  A = 3902.229   WP Peak Temp = 92.006
[6.37372079 6.37372079]
iter 5;   DS = 6.374  WPS = 6.374  A = 2985.473   WP Peak Temp = 100.854
[7.04464923 5.63219869]
iter 6;   DS = 7.045  WPS = 5.632  A = 2937.840   WP Peak Temp = 101.462
[6.46225458 6.46225458]
iter 7;   DS = 6.462  WPS = 6.462  A = 3063.397   WP Peak Temp = 99.938
[6.55064293 6.37386623]
iter 8;   DS = 6.551  WPS = 6.374  A = 3064.784   WP Peak Temp = 99.942
[6.45120603 6.45120603]
iter 9;   DS = 6.451  WPS = 6.451  A = 3053.618   WP Peak Temp = 100.051
[6.44015749 6.46225458]
iter 10;  DS = 6.440  WPS = 6.462  A

iter 2;   DS = 5.161  WPS = 5.161  A = 7435.584   WP Peak Temp = 105.657
[6.16088433 5.16088433]
iter 3;   DS = 6.161  WPS = 5.161  A = 8836.770   WP Peak Temp = 98.484
[5.16088433 6.16088433]
iter 4;   DS = 5.161  WPS = 6.161  A = 8794.082   WP Peak Temp = 98.484
[5.55523717 5.55523717]
iter 5;   DS = 5.555  WPS = 5.555  A = 8563.685   WP Peak Temp = 99.785
[5.64362552 5.46684882]
iter 6;   DS = 5.644  WPS = 5.467  A = 8565.458   WP Peak Temp = 99.789
[5.46684882 5.46684882]
iter 7;   DS = 5.467  WPS = 5.467  A = 8303.915   WP Peak Temp = 101.027
[5.53993798 5.53993798]
iter 8;   DS = 5.540  WPS = 5.540  A = 8518.435   WP Peak Temp = 99.997
[5.55098652 5.52888944]
iter 9;   DS = 5.551  WPS = 5.529  A = 8518.875   WP Peak Temp = 99.997
[5.53640245 5.53640245]
iter 10;  DS = 5.536  WPS = 5.536  A = 8507.995   WP Peak Temp = 100.046
3 fuel unit footprint simulated
No solution; single package violates constraint
No solution; single package violates constraint
No solution; single package v

In [ ]:
# Save results of footprint minimizations at each storage time
for st in sts:
    df_min = get_min_areas(reactors, workspace_path, st)
    df_min.to_csv(workspace_path + 'results/footprint_minimization/min_footprint_' + str(int(st)) + '.csv', index = False)